# Problem 1 - Comparing thermochronometer ages (*12 points*)

## Overview

![Bhutan Himalaya](img/Bhutan_Himalaya.png)<br/>
*Figure 1. The Himalaya in Bhutan. [Image source](http://commons.wikimedia.org/wiki/File:View_of_Gasa_Dzong.JPG).*

The overall goal of this exercise is to use our heat transfer model to interpret [a thermochronometer dataset from the Himalaya of Bhutan](data/Bhutan_age_data.txt). The interpretation entails determining long-term average rock exhumation rates from rock samples analyzed using apatite and zircon (U-Th)/He, and muscovite <sup>40</sup>Ar/<sup>39</sup>Ar thermochronology. As you will recall, [our exercise last week](https://github.com/IntroQG-2019/Exercise-6) used a 1-D time-dependent solution to the advection-diffusion equation to calculate a temperature-depth profile in the Earth, which was then used to predict thermochronometer ages based on Dodson's method. In that model we specified a rock advection velocity and observed variations in the thermochronometer ages as a function of this  velocity, also known as the exhumation rate. This week we will compare those predicted thermochronometer ages to data from Bhutan with the goal of minimizing the misfit between the measured and predicted thermochronometer ages by varying the specified rock advection velocity (exhumation rate), which will allow us to define a best-fit exhumation rate (or exhumation history) for the Himalaya of Bhutan. For this exercise, we will be using data from [Coutand et al., 2014](https://dx.doi.org/10.1002/2013JB010891) and [Stüwe and Foster, 2001](https://www.sciencedirect.com/science/article/pii/S1367912000000183) (PDFs available on the [main course page](https://introqg.github.io/site/final-paper/articles.html)).

In this problem you will modify the Python cells below to read in a data file of thermochronometer data and compare the data to predictions from the simple numerical model used in [Exercise 6](https://github.com/IntroQG-2019/Exercise-6). After making these changes you will modify the plotting to shown not only the the geotherm and particle cooling history used in Exercise 6, but also the age data from the data file and the predicted thermochronometer ages.

**Notice**: Closely follow the instructions! For example, you should be sure to use **exactly** the same variable names mentioned in the instructions because your answers will be automatically graded, and the tests that grade your answers rely on following the same formatting or variable naming as in the instructions.

**Your score on this problem will be based on following criteria**:

- Modifying the provided Python function to format and add text to the plots
- Plotting your results and adding figure captions
- Including comments that explain what most lines in the code do
- Uploading your notebook to your GitHub repository for this week's exercise

## Part 1: Redoing Exercise 6 (*2 points*)

The first task in this problem is to edit the Python cell below to contain the changes you were asked to make in [Exercise 6](https://github.com/IntroQG-2019/Exercise-6). Basically, you have the same Python cells we started with last week in Exercise 6, and we want to keep the changes made in that exercise by editing the cell below.
Note that only one cell need to be edited (the one that is not read-only).

- Run the first *four* Python cells below.
- Edit the cell containing the `age_predict` function to do the following things from [Exercise 6](https://github.com/IntroQG-2019/Exercise-6):
    - Add axis labels and a title to the plot it produces.
    - Add the advection velocity to your the plot using the `ax.text()` function.
    - Display the predicted apatite (U-Th)/He, zircon (U-Th)/He, and muscovite $^{40}$Ar/$^{39}$Ar ages and predicted closure temperatures on the plot using the `ax.text()` function.
        - As before, I suggest that you use the text plotting function near the bottom of the `age_predict` function where the predicted ages are printed to the screen (if requested).
    - Enable plotting of the exhumation history.
- Run the Python cell containing the `age_predict` function to store the updated version in memory.
- Call the `age_predict` function with calculation of the all three thermochronometer ages enabled.
    - As before, look at the definition of `age_predict` to see how to set the flags (`True`/`False` parameters) that allow you to enable the calculation of different thermochronometers.
    - This should reproduce Figure 9 from Exercise 6.
- Add a figure caption for the plot in the cell beneath it describing it as if it were in a scientific journal article.

In [ ]:
# Configure Matplotlib plotting
%matplotlib inline

In [ ]:
# Import modules
import numpy as np
import pandas as pd
from scipy.special import erfc
import matplotlib.pyplot as plt

In [ ]:
def temp_1D(z, time, g_initial, vz, kappa):
    """Calculate solution to 1D transient advection-diffusion heat transfer equation.
    
    Keyword arguments:
    z -- array of depths from surface (units: km)
    time -- array of time since model start (units: m)
    g_initial -- initial temperature gradient (units: deg. C / km)
    vz -- vertical advection velocity (units: km / Ma)
    kappa -- thermal diffusivity (units: km^2 / Ma)
    """
    # Calculate T separately for case where t = 0 to avoid divide-by-zero warnings
    if time == 0:
        temperature = g_initial * z
    else:
        temperature = g_initial * (z + vz * time) + (g_initial / 2.0) * ((z - vz * time) *\
                      np.exp(-(vz * z) / kappa) * erfc((z - vz * time) / (2.0 * np.sqrt(kappa *time))) -\
                      (z + vz * time) * erfc((z + vz * time) / (2.0 * np.sqrt(kappa * time))))
    return temperature

In [ ]:
def dodson(tau, temp_hist, temp_hist_prev, time, time_prev, age, tc_prev, ea, dtdt, a, d0a2, r):
    """Calculate thermochronometer age and closure temperature using Dodson's method.
    
    Keyword arguments:
    tau -- diffusivity characteristic time (units: Ma)
    temp_hist -- current temperature in history (units: deg. C)
    temp_hist_prev -- temperature from last iteration (units: deg. C)
    time -- time until end of simulation (units: Ma)
    time_prev -- previous time until end of simulation (units: Ma)
    age -- previous calculated thermochronometer age (units: Ma)
    tc_prev -- previous calculated closure temperature (units: deg. C)
    ea -- activation energy (units: J / mol)
    dtdt -- cooling rate (units: deg. C / Ma)
    a -- geometric factor (25 for a sphere, 8.7 for a planar sheet)
    d0a2 -- diffusivity at infinite temperature over domain squared (units: 1 / s)
    r -- universal gas constant (units: J / (mol K))
    """
    # Calculate diffusivity characteristic time
    tau = (r * (temp_hist + 273.15)**2.0) / (ea * dtdt)
    # Calculate new closure temperature
    tc = ea / (r * np.log(a * tau * d0a2)) - 273.15
    # Calculate new cooling age if temperature is above tc
    if temp_hist > tc:
        ratio = (tc_prev - temp_hist_prev)/(tc_prev - temp_hist_prev + temp_hist - tc)
        age = time + (time_prev - time) * ratio
    tc_prev = tc
    return age, tc, tc_prev

Edit only the Python cell below.

In [ ]:
def age_predict(temp_gradient=10.0, time_total=50.0, vz=0.5,
                plot_temp_z_hist=False, n_temp_save=1,
                calc_ahe=False, calc_zhe=False, calc_mar=False):
    """Calculate transient 1D thermal solution and predict thermochronometer ages.
    
    Keyword arguments:
    temp_gradient -- initial thermal gradient (units: deg. C / km; default: 10.0)
    time_total -- total thermal model simulation time (units: Ma; default: 50.0)
    vz -- vertical advection velocity (units: km/Ma; default: 0.5)
    plot_temp_z_hist -- save the temperature-depth history of the tracked particle (default: False)
    n_temp_save -- number of temperature profiles to dave (default: 1)
    calc_ahe -- calculate apatite (U-Th)/He age (default: False)
    calc_zhe -- calculate zircon (U-Th)/He age (default: False)
    calc_mar -- calculate muscovite 40Ar/39Ar age (default: False)
    """

    # THERMAL MODEL PARAMETERS
    z_max = 50.0            # Thermal model thickness [km]
    kappa = 32.0            # Thermal diffusivity [km2 / Ma]
    npz = 101               # Number of depth points for temperature calculation
    npt = 401               # Number of times for temperature calculation
    high_temp = 1000.0      # Temperature to assign if tracked particle depth exceeds zmax
    
    # CLOSURE TEMPERATURE PARAMETERS
    # Apatite (U-Th)/He
    size_ahe = 100.0        # Apatite grain size [um]
    ea_ahe = 138.0e3        # Activation energy [J/mol]
    a_ahe = 25.0            # Geometry factor [25 for sphere]
    d0_ahe = 5.0e-3         # Diffusivity at infinite temperature [m2/s]
    tau_ahe = 1.0           # Initial guess for characteristic time

    # Zircon (U-Th)/He
    size_zhe = 100.0        # Zircon grain size [um]
    ea_zhe = 168.0e3        # Activation energy [J/mol]
    a_zhe = 25.0            # Geometry factor [25 for sphere]
    d0_zhe = 4.6e-5         # Diffusivity at infinite temperature [m2/s]
    tau_zhe = 1.0           # Initial guess for characteristic time

    # Muscovite Ar/Ar
    size_mar = 500.0        # Muscovite grain size [um]
    ea_mar = 183.0e3        # Activation energy [J/mol]
    a_mar = 8.7             # Geometry factor [8.7 for planar sheet]
    d0_mar = 3.3e-6         # Diffusivity at infinite temperature [m2/s]
    tau_mar = 1.0           # Initial guess for characteristic time

    # OTHER CONSTANTS
    r = 8.314               # Universal gas constant

    # Set initial thermochronometer ages
    age_ahe = time_total
    age_zhe = time_total
    age_mar = time_total

    # Convert units
    size_ahe = size_ahe / 1.0e6 / 1.0e3                                     # um -> km
    size_zhe = size_zhe / 1.0e6 / 1.0e3                                     # um -> km
    size_mar = size_mar / 1.0e6 / 1.0e3                                     # um -> km
    d0_ahe = d0_ahe * (1 / 1000.0**2.0) * (1.0e6 * 365.25 * 24.0 * 3600.0)  # m2/s -> km2/Ma
    d0_zhe = d0_zhe * (1 / 1000.0**2.0) * (1.0e6 * 365.25 * 24.0 * 3600.0)  # m2/s -> km2/Ma
    d0_mar = d0_mar * (1 / 1000.0**2.0) * (1.0e6 * 365.25 * 24.0 * 3600.0)  # m2/s -> km2/Ma

    # Calculate diffusion parameter D0/a2 for each mineral
    d0a2_ahe = d0_ahe / size_ahe**2.0    # Apatite
    d0a2_zhe = d0_zhe / size_zhe**2.0    # Zircon
    d0a2_mar = d0_mar / size_mar**2.0    # Muscovite

    # Create thermal model arrays
    z = np.linspace(0.0, z_max, npz)                    # Define depth range array
    time = np.linspace(0.0, time_total, npt)            # Define time range array
    time_ma = np.linspace(time_total, 0.0, npt)         # Define time array in Ma (time before present)
    z_hist = np.linspace(time_total*vz, 0.0, npt)       # Define z particle position history
    temp_hist = np.zeros(npt)                           # Define initial temperature history array
    temp = np.zeros(npz)                                # Define initial temperature array

    # Define increment for saving temperatures
    iout = 0

    # Create temperatures DataFrame
    temp_df = pd.DataFrame(data={'Depth (km)':-z})

    # Loop over all times and calculate temperature
    for i in range(npt):
        # Calculate temperature at time[i]
        temp = temp_1D(z, time[i], temp_gradient, vz, kappa)
        
        # Set the temperature history temperature to high_temp if the tracked particle
        # depth is below z_max (where temperature would be undefined)
        if z_hist[i] > max(z):
            temp_hist[i] = high_temp
        # Otherwise, store the current temperature at the depth of the tracked particle
        else:
            temp_hist[i] = temp_1D(z_hist[i], time[i], temp_gradient, vz, kappa)

        # If the current iteration is one of the save increments, save the geotherm
        if i == iout:
            string = str(time_ma[i])+" Ma"
            temp_df[string] = temp
            iout = iout + int(float(len(time)-1)/float(n_temp_save))

    # Set the initial closure temperatures for the previous step to one
    tc_ahe_prev = 1.0
    tc_zhe_prev = 1.0
    tc_mar_prev = 1.0

    # Set the previous temperature to the max value in the current temperature array
    temp_hist_prev = max(temp)

    # Loop over all positions in the temperature history array
    for i in range(len(temp_hist)):
        # Calculate the cooling rate for the first temperature value
        if i == 0:
            dtdt = (temp_hist[i] - temp_hist[i+1]) / (time_ma[i] - time_ma[i+1])
        # Calculate the cooling rate for the last temperature value
        elif i == len(temp_hist)-1:
            dtdt = (temp_hist[i-1] - temp_hist[i]) / (time_ma[i-1] - time_ma[i])
        # Calculate the cooling rate for the the intermediate temperature values
        else:
            dtdt = (temp_hist[i-1] - temp_hist[i+1]) / (time_ma[i-1] - time_ma[i+1])

        # Ensure the cooling rate is at least 1 deg. C per 10 Ma
        dtdt = max(dtdt, 0.1 / (1.0e6 * 365.25 * 24.0 * 3600.0))

        # Calculate various closure temperatures, if requested
        if calc_ahe:
            age_ahe, tc_ahe, tc_ahe_prev = dodson(tau_ahe, temp_hist[i], temp_hist_prev, time_ma[i],
                                                  time_ma[i-1], age_ahe, tc_ahe_prev, ea_ahe, dtdt,
                                                  a_ahe, d0a2_ahe, r)
        if calc_zhe:
            age_zhe, tc_zhe, tc_zhe_prev = dodson(tau_zhe, temp_hist[i], temp_hist_prev, time_ma[i],
                                                  time_ma[i-1], age_zhe, tc_zhe_prev, ea_zhe, dtdt,
                                                  a_zhe, d0a2_zhe, r)
        if calc_mar:
            age_mar, tc_mar, tc_mar_prev = dodson(tau_mar, temp_hist[i], temp_hist_prev, time_ma[i],
                                                  time_ma[i-1], age_mar, tc_mar_prev, ea_mar, dtdt,
                                                  a_mar, d0a2_mar, r)

        # Store previous temperature in thermal history
        temp_hist_prev = temp_hist[i]

    # Save particle depth-temperature history if requested
    if plot_temp_z_hist:
        ex_hist_df = pd.DataFrame(index=temp_hist, data={'Exhumation history':-z_hist})
    else:
        ex_hist_df = None

    # Plot results
    # Make list of column titles, excluding depth
    cols = temp_df.columns.values[1:]
    # Plot starting geotherm
    ax = temp_df.plot(x=cols[0], y='Depth (km)', figsize=(10,7), label=cols[0])
    # Loop through and plot requested geotherms
    for column in cols[1:]:
        temp_df.plot(x=column, y='Depth (km)', ax=ax, label=column)
    # Plot exhumation history, if requested
    if plot_temp_z_hist:
        # Delete "pass" below and add your commands to plot the exhumation history
        pass

    # Write thermochronometer age(s) to screen, if requested
    # Set to ages to None if age prediction is not requested
    if calc_ahe:
        print("Apatite (U-Th)/He age: {0:.2f} Ma".format(age_ahe))
    else:
        age_ahe = None

    if calc_zhe:
        print("Zircon (U-Th)/He age: {0:.2f} Ma".format(age_zhe))
    else:
        age_zhe = None

    if calc_mar:
        print("Muscovite Ar/Ar age: {0:.2f} Ma".format(age_mar))
    else:
        age_mar = None
    
    return temp_df, ex_hist_df, ax, age_ahe, age_zhe, age_mar

Call the updated `age_predict` function in the cell below.

In [ ]:
# Call the age_predict function below
age_output = None

# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

In [ ]:
# This test should work
print('The zircon (U-Th)/He age should be 26.84 Ma. My age is {0:.2f} Ma.'.format(age_output[4]))


## Part 2: Reading the age data (*2 points*)

In order to be able to compare our predicted thermochronometer ages to some data, we'll next need to load [the data file](data/Bhutan_age_data.txt). Using the past exercises as examples (look at those notebooks :) ), you should edit the `age_predict` function cell to read a data file and store the contents in a pandas DataFrame. The data file has a header that lists the data contained in each column, and there are NA values indicated by `'-9999'`.

- Add some lines to the `age_predict` function to read [the data file](data/Bhutan_age_data.txt) into a variable called `data` using pandas.
    - When you read in the data, also convert the `'-9999'` values to NA values in the DataFrame.
        - Measured ages are listed for each sample location in the data file, but not every sample has been analyzed for each different thermochronometer system.
    - I suggest you read the data file after the predicted thermochronometer ages are calculated.
- Create **3** separate pandas Series for the measured age data without the NA values.
    - These could be called `ahe_data`, `zhe_data`, and `mar_data`, for example and would only contain the measured age values for the corresponding age systems (i.e., no missing data in these Series).
    - If you're having trouble, check out [the hints for this week's exercise](https://introqg.github.io/site/lessons/L7/exercise-7.html)!
- Run the Python cell containing the `age_predict` function to store the updated version in memory.
- Call the `age_predict` function in the cell below just to make sure it works. There is no need to save the plot or include a caption for this part.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Part 3: Calculating a goodness-of-fit, again (*3 points*)

Your next task is to define a function to calculate the goodness-of-fit using the reduced chi-squared equation introduced in Exercise 2,

\begin{equation}
  \Large
  \chi^{2} = \frac{1}{N} \sum \frac{(O_{i} - E_{i})^{2}}{\sigma_{i}^2}
\end{equation}

where $N$ is the number of ages, $O_{i}$ is the $i$th measured age, $E_{i}$ is the $i$th predicted age, and $\sigma_{i}$ is the $i$th standard deviation.

- Create a function called `chi_squared` that can be used to calculate the reduced chi-squared value for the data in the age data file.
    - **NOTE**: We will be using this reduced chi-squared equation to calculate the misfit between each measured age and a single predicted age for a given thermochronometer system. Thus, when you create your `chi_squared` function it should assume you call it with a list or array of measured ages and a single predicted age (rather than a list of predicted ages).
    - If you're having trouble, check out [the hints for this week's exercise](https://introqg.github.io/site/lessons/L7/exercise-7.html)!
- Modify the cell containing the `age_predict` function to call your `chi_squared` function any time predicted ages are calculated for one of the thermochronometer systems.
    - This should be done below the part of the function that reads the data file.
    - Store the misfit values as `ahe_misfit`, `zhe_misfit`, and `mar_misfit`.
    - You should also add the `ahe_misfit`, `zhe_misfit`, and `mar_misfit` variables (in that order) to the list of values returned from the function.
    - Remember to use the age and uncertainty values without NA values when calculating the misfits.
        - If you're having trouble with this, check out [the hints for this week's exercise](https://introqg.github.io/site/lessons/L7/exercise-7.html)!
- Run the Python cell containing the `age_predict` function to store the updated version in memory.
- Call the `age_predict` function in the cell below the `chi_squared` function tests just to make sure it works. There is no need to save the plot or include a caption for this part.

In [ ]:
def chi_squared(measured, predicted, std):
    """Returns the reduced chi-squared value for input age data."""
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# The visible tests below should work
from nose.tools import assert_equal

# Fake data
obs = np.array([1.1, 2.9, 2.6, 3.5, 5.7, 2.8])
pred = np.array(1.5)
std = np.array([0.5, 1.4, 0.6, 1.5, 0.7, 1.0])

# Print calculated misfit value
print('The goodness-of-fit should be 7.41. My goodness-of-fit is {0:.2f}.'.format(chi_squared(obs, pred, std)))

# Check that the function values are correct
assert_equal(round(chi_squared(obs[:3], pred, std[:3]), 4), 1.6670)
assert_equal(round(chi_squared(obs, pred, std), 4), 7.4115)

Run the `age_predict` function below to test that the `chi_squared` function is working properly within it.

In [ ]:
# Call the age_predict function below
age_output = None

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# This test should work
print('The apatite (U-Th)/He misfit should be 287.99. My age is {0:.2f}.'.format(age_output[6]))


In [ ]:
# This test should work
print('The zircon (U-Th)/He misfit should be 23906.57. My age is {0:.2f}.'.format(age_output[7]))


## Part 4: Visualizing the data fit (*5 points*)

The last task in this problem is to produce a useful plot.

- Edit the cell with the `age_predict` function to:
    - Use the `plt.subplots()` command to make a plot that shows (a) the predicted geotherm and particle temperature-depth history in the upper panel (i.e., the plot from Exercise 6), and (b) the predicted and measured age data on the lower panel.
        - This can be done by adding the command `fig, (ax, ax2) = plt.subplots(nrows=2, ncols=1)` where above the location where the first plot is generated. This command will create 2 subplots that can be referenced as `ax` and `ax2`.
        - **Note**: With this change you will no longer define the value of `ax` when creating the first plot, but rather add the parameter `ax=ax` to the pandas plot function when making the first plot.
    - For the lower plot:
        - The *x*-axis should plot latitude and the *y*-axis should plot thermochronometer age, including vertical error bars for the measured ages.
            - You can use the full measured age arrays here, not the "clean" data Series since the NA values will not be displayed on the plot.
        - Plot the different thermochronometer systems with different symbol colors.
        - The predicted ages can be shown using lines of constant age that have a range that includes the entire range of latitude of the measured age data. Their colors should match the corresponding measured age colors.
        - This plot should also list the goodness-of-fit values for each thermochronometer system, as well as a total goodness-of-fit for the age dataset. You can use the `ax2.text()` function for this.
        - Be sure to also define the axis labels.
- Re-run the Python cell containing the `age_predict` function to store the updated version in memory.
- Call the age_predict function with calculation of the all three thermochronometer ages **and** the time-temperature exhumation history enabled.
    - As above, look at the definition of `age_predict` to see how to set the flags (`True`/`False` parameters) that allow you to enable the calculation of different thermochronometers.
- Add a figure caption for the plot in the cell beneath it describing it as if it were in a scientific journal article.

In [ ]:
# Call the age_predict function below
age_output = None

# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

# Problem 2 - "Fitting" thermochronometer data (*8 points*)

Using the code changes you have made in Problem 1, your goal is now to find the average long-term exhumation rates that provide good fits to the measured thermochronometer data.

## Part 1: Fitting all the ages (*2 points*)

We'll start this problem by minimizing the misfit value for all of the thermochronometer data in this part.

- Call the `age_predict` function in a series of models where you change only the vertical advection velocity `vz_all` in order to find a minimum goodness-of-fit value for the whole thermochronometer age dataset (i.e., the total misfit).
    - Call the age_predict function with calculation of the all three thermochronometer ages **and** the time-temperature exhumation history enabled.
        - As above, look at the definition of `age_predict` to see how to set the flags (`True`/`False` parameters) that allow you to enable the calculation of different thermochronometers.
    - You do not need to find the absolute minimum goodness-of-fit value, but rather the minimum value you get for advection velocities to the nearest 0.1 km/Ma.
- Add a figure caption for the plot in the cell beneath it describing it as if it were in a scientific journal article.
    - Be sure the advection velocity (exhumation rate) and misfit value are clearly displayed on the plot.

In [ ]:
# Call the age_predict function below
vz_all = None
age_output = None

# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

In [ ]:
# This test should work
print('My predicted apatite (U-Th)/He age for an exhumation rate of {0:.1f} km/Ma is {1:.2f} Ma.'.format(vz_all, age_output[3]))


## Part 2: Fitting the apatite (U-Th)/He ages (*2 points*)

Now we can focus on fitting a single thermochronometer system.

- Call the `age_predict` function in a series of models where you change only the vertical advection velocity `vz_ahe` in order to find a minimum goodness-of-fit value to the apatite (U-Th)/He age data.
    - Call the age_predict function with calculation of the all three thermochronometer ages **and** the time-temperature exhumation history enabled.
        - As above, look at the definition of `age_predict` to see how to set the flags (`True`/`False` parameters) that allow you to enable the calculation of different thermochronometers.
    - You do not need to find the absolute minimum goodness-of-fit value, but rather the minimum value you get for advection velocities to the nearest 0.1 mm/a.
- Add a figure caption for the plot in the cell beneath it describing it as if it were in a scientific journal article.
    - Be sure the advection velocity (exhumation rate) and apatite (U-Th)/He misfit value are clearly displayed on the plot.

In [ ]:
# Call the age_predict function below
vz_ahe = None
age_output = None

# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

In [ ]:
# This test should work
print('My predicted apatite (U-Th)/He age for an exhumation rate of {0:.1f} km/Ma is now {1:.2f} Ma.'.format(vz_ahe, age_output[3]))


## Part 3: Fitting the zircon (U-Th)/He ages (*2 points*)

Now we can focus on fitting a single thermochronometer system.

- Call the `age_predict` function in a series of models where you change only the vertical advection velocity `vz_zhe` in order to find a minimum goodness-of-fit value to the zircon (U-Th)/He age data.
    - Call the age_predict function with calculation of the all three thermochronometer ages **and** the time-temperature exhumation history enabled.
        - As above, look at the definition of `age_predict` to see how to set the flags (`True`/`False` parameters) that allow you to enable the calculation of different thermochronometers.
    - You do not need to find the absolute minimum goodness-of-fit value, but rather the minimum value you get for advection velocities to the nearest 0.1 mm/a.
- Add a figure caption for the plot in the cell beneath it describing it as if it were in a scientific journal article.
    - Be sure the advection velocity (exhumation rate) and zircon (U-Th)/He misfit value are clearly displayed on the plot.

In [ ]:
# Call the age_predict function below
vz_zhe = None
age_output = None

# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

In [ ]:
# This test should work
print('My predicted zircon (U-Th)/He age for an exhumation rate of {0:.1f} km/Ma is {1:.2f} Ma.'.format(vz_zhe, age_output[4]))


## Part 4: Fitting the muscovite $^{40}$Ar/$^{39}$Ar ages (*2 points*)

Now we can focus on fitting a single thermochronometer system.

- Call the `age_predict` function in a series of models where you change only the vertical advection velocity `vz_mar` in order to find a minimum goodness-of-fit value to the muscovite $^{40}$Ar/$^{39}$Ar age data.
    - Call the age_predict function with calculation of the all three thermochronometer ages **and** the time-temperature exhumation history enabled.
        - As above, look at the definition of `age_predict` to see how to set the flags (`True`/`False` parameters) that allow you to enable the calculation of different thermochronometers.
    - You do not need to find the absolute minimum goodness-of-fit value, but rather the minimum value you get for advection velocities to the nearest 0.1 mm/a.
- Add a figure caption for the plot in the cell beneath it describing it as if it were in a scientific journal article.
    - Be sure the advection velocity (exhumation rate) and muscovite $^{40}$Ar/$^{39}$Ar misfit value are clearly displayed on the plot.

In [ ]:
# Call the age_predict function below
vz_mar = None
age_output = None

# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

In [ ]:
# This test should work
print('My predicted muscovite Ar/Ar age for an exhumation rate of {0:.1f} km/Ma is {1:.2f} Ma.'.format(vz_mar, age_output[5]))


## References

Coutand, I., Whipp, D. M., Grujic, D., Bernet, M., Fellin, M. G., Bookhagen, B., et al. (2014). [Geometry and kinematics of the Main Himalayan Thrust and Neogene crustal exhumation in the Bhutanese Himalaya derived from inversion of multithermochronologic data](https://dx.doi.org/10.1002/2013JB010891). *Journal of Geophysical Research: Solid Earth*.

Stüwe, K., & Foster, D. (2001). [<sup>40</sup>Ar/<sup>39</sup>Ar, pressure, temperature and fission track constraints on the age and nature of metamorphism around the main central thrust in the eastern Bhutan Himalaya](https://www.sciencedirect.com/science/article/pii/S1367912000000183). *Journal of Asian Earth Sciences*, 19(1), 85–95.